<a href="https://colab.research.google.com/github/AIS-SG/ai_lab_224n_NLP/blob/main/%EC%98%81%EC%96%B4_%ED%95%9C%EA%B5%AD%EC%96%B4_Word2Vec_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 딥러닝을 이용한 자연어 처리 입문 - 영어/한국어 Word2Vec 실습
#### 참고 자료 : https://wikidocs.net/50739

# 1. 영어 Word2Vec 만들기

### 1) 훈련 데이터 이해하기
##### 얻고자 하는 실질적 데이터는 영어 문장으로만 구성된 <context>와 </context> 사이의 내용이다.
##### (Laughter)나 (Applause)와 같은 배경음을 나타내는 단어들을 제거해야 한다.

In [ ]:
import re
import lxml.etree
import urllib.request
import zipfile
from nltk.tokenize import word_tokenize, sent_tokenize

### 2-1) 데이터 로드

In [ ]:
import nltk.data
nltk.download('punkt')

In [ ]:
urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")
# 데이터 다운로드

with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
  target_text = etree.parse(z.open('ted_en-20160408.xml', 'r'))
  parse_text = '\n'.join(target_text.xpath('//content/text()'))
# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.

In [ ]:
# 로드된 데이터 중에서 300개의 글자만 출력
parse_text[:300]

### 2-2) 전처리 수행

In [ ]:
content_text = re.sub(r'\([^)]*\)', '', parse_text)
# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.

sent_text=sent_tokenize(content_text)
# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.

normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.

result = [word_tokenize(sentence) for sentence in normalized_text]
# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.

In [ ]:
print('총 샘플의 개수 : {}'.format(len(result)))

In [ ]:
for line in result[:3]: # 샘플 3개만 출력
  print(line) 

### 3) Word2Vec 훈련시키기

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)



1.   size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원
2.   window = 컨텍스트 윈도우 크기
3.   min_count = 단어 최소 빈도 수 제한(빈도가 적은 단어들은 학습하지 않는다.)
4.   workers = 학습을 위한 프로세스 수
5.   sg = 0은 CBOW, 1은 Skip-gram.



In [ ]:
# man과 가장 유사한 단어들 출력
model_result = model.wv.most_similar("man")
print(model_result)

### 4) Word2Vec 모델 저장하고 로드하기

In [ ]:
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('eng_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드

In [ ]:
# 로드한 모델에서 다시 man과 유사한 단어를 출력
model_result = loaded_model.most_similar("man")
print(model_result)